# 17. Frame Selected Discriminality and ICC

In [1]:
from src import *

In [2]:
import pingouin

## Description

We evaluate frame selected's discriminality and ICC values. 

GOAL: Compute frame selected's discriminality and ICC values. 

## Table of Contents
- [Frame selected and discrim / ICC](#comparison)

<a class="anchor" id="comparison"></a>
## Frame selected (by emotion) gradients

In [3]:
# Params
embedding  = "dm" # diffusion map
aff_kernel = "pearson" #affinity matrix kernel
align_meth = "procrustes"
n_iter     = 10 # procrustes align number of iteration
nb_comp    = 3
rs         = 99

D = lambda x,y: ((x-y)**2).sum(axis=1)

Load frame selected and full movie gradients (individual level)

In [82]:
total_series = load('resources/threshes/tresh0-1.pkl')
# select with thresholded frame selected gradients 
fselected_series = load('resources/threshes/tresh0-1.pkl') #take top 80 frames
# fselected_series = load('resources/threshes/tresh80-1.pkl') #take top 20 frames

del total_series['Rest']
del fselected_series['Rest']

In [83]:
f_selected_gradients = {k:{} for k in fselected_series}

for mname in fselected_series:
    tmp_dict = fselected_series[mname]
#     perc_left = tmp_dict['S15'].shape[0]/total_series[mname]['S15'].shape[0]
#     print('Percentage of original series left {}'.format(np.round(perc_left,3)))
    for _id in tmp_dict:
        tmp_series = tmp_dict[_id]
        fc_selects = FC(tmp_series)
        g_selects = GradientMaps(n_components=nb_comp, approach=embedding, 
                            kernel=aff_kernel, random_state=rs)
        g_selects.fit(fc_selects)
        grad = g_selects.gradients_
        fscore = D(grad.mean(axis=0), grad)
        f_selected_gradients[mname][_id] = fscore

In [84]:
# compute randomly picked timeseries of same length
np.random.seed(19)
full_movie_grads = {k:{} for k in fselected_series}

for mname in fselected_series:
    perc_left = fselected_series[mname]['S15'].shape[0]
    tmp_dict = total_series[mname]
    
    for _id in tmp_dict:
        tmp_series = tmp_dict[_id]
        tmp_index = np.random.choice(np.arange(len(tmp_series)), perc_left,replace=False)
        tmp_series = tmp_series[tmp_index]
        
        fc_selects = FC(tmp_series)
        g_selects = GradientMaps(n_components=nb_comp, approach=embedding, 
                            kernel=aff_kernel, random_state=rs)
        g_selects.fit(fc_selects)
        grad = g_selects.gradients_
        fscore = D(grad.mean(axis=0), grad)
        full_movie_grads[mname][_id] = fscore        

Compute ICC across movies using class as the ID

In [85]:
# all_gradients = {}
# for mname in f_selected_gradients:
#     if 'Rest' in mname: continue
#     all_gradients[mname+'_fselect'] = f_selected_gradients[mname]
# for mname in full_movie_grads:
#     all_gradients[mname] = full_movie_grads[mname]

We want to know the reliability of 

In [121]:
df = {'movieid': [], 'id': [], 'fscore': []}
df2 = {'movieid': [], 'id': [], 'fscore': []}

idx = 2
for mname in np.array(list(f_selected_gradients.keys()))[[3,6]]:
    for _id in f_selected_gradients[mname]:
        df['id'] += [int(_id[1:])]
        df['fscore'] += [f_selected_gradients[mname][_id][idx]]
        df['movieid'] += [mname]
        
        df2['id'] += [int(_id[1:])]
        df2['fscore'] += [full_movie_grads[mname][_id][idx]]
        df2['movieid'] += [mname]        

df = pd.DataFrame.from_dict(df)
df2 = pd.DataFrame.from_dict(df2)

icc = pingouin.intraclass_corr(data=df, targets='id', raters='movieid', ratings='fscore')
print(np.round(list(icc['ICC']),3))
icc2 = pingouin.intraclass_corr(data=df2, targets='id', raters='movieid', ratings='fscore')
print(np.round(list(icc2['ICC']),3))
print()

[0.5   0.513 0.541 0.667 0.678 0.702]
[0.5   0.513 0.541 0.667 0.678 0.702]



In [110]:
for idx in range(0,414):
    df = {'movieid': [], 'id': [], 'fscore': []}
    df2 = {'movieid': [], 'id': [], 'fscore': []}
    for mname in f_selected_gradients:
        for _id in f_selected_gradients[mname]:
            df['id'] += [int(_id[1:])]
            df['fscore'] += [f_selected_gradients[mname][_id][idx]]
            df['movieid'] += [mname]

            df2['id'] += [int(_id[1:])]
            df2['fscore'] += [full_movie_grads[mname][_id][idx]]
            df2['movieid'] += [mname]        

    df = pd.DataFrame.from_dict(df)
    df2 = pd.DataFrame.from_dict(df2)

    print('vertex {}'.format(idx))
    icc = pingouin.intraclass_corr(data=df, targets='id', raters='movieid', ratings='fscore')
    print(np.round(list(icc['ICC']),3))
    icc2 = pingouin.intraclass_corr(data=df2, targets='movieid', raters='id', ratings='fscore')
    print(np.round(list(icc2['ICC']),3))